# Train finale FashionCNN e salvataggio modello

In [ ]:

import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

class FashionCNN(nn.Module):
    def __init__(self, hidden_units=400):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,3,padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,3), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(64*6*6, hidden_units)
        self.fc2 = nn.Linear(hidden_units, 120)
        self.fc3 = nn.Linear(120, 10)
    def forward(self, x):
        x = self.layer1(x); x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x); x = self.fc2(x); x = self.fc3(x)
        return x

def train_final_model(lr=4.8e-4, batch_size=64, hidden=400, epochs=3, out_path="models/fashion_cnn.pth"):
    tfm = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    ds = datasets.FashionMNIST(root=".data", train=True, download=True, transform=tfm)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)
    model = FashionCNN(hidden_units=hidden)
    crit = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=lr)
    for ep in range(epochs):
        model.train(); running = 0.0
        for x,y in dl:
            opt.zero_grad(); loss = crit(model(x), y); loss.backward(); opt.step(); running += loss.item()
        print(f"Epoch {ep+1}, Loss: {running/len(dl):.4f}")
    os.makedirs("models", exist_ok=True)
    torch.save(model.state_dict(), out_path)
    print(f"Model salvato in {out_path}")
    return out_path

_ = train_final_model()
